In [1]:
from tomocupy_stream import GPURecRAM
import numpy as np
import tifffile
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import matplotlib.pyplot as plt
%matplotlib inline

# Download data with the direct link

In [2]:
!wget -nc https://anl.box.com/shared/static/or8vlzdu07d8zwxvk50ihwghq39ide0o.npz

File ‘or8vlzdu07d8zwxvk50ihwghq39ide0o.npz’ already there; not retrieving.



# Read data, dark and flat fields, projection angles, the rotation center

In [3]:
mdict = np.load('or8vlzdu07d8zwxvk50ihwghq39ide0o.npz')
data = mdict['data']
dark = mdict['dark']
flat = mdict['flat']
theta = mdict['theta']
center = mdict['center']


# Data consists of 1501 projections of the size [1024x1360]. For convience data are saved as sinograms (first two dimensions are swapped)

In [4]:
data.shape 

(1024, 751, 1360)

# Plot projections

In [5]:
def plot_projections(sid=1):
    plt.imshow(data[:,sid],cmap='gray')
interact(plot_projections, sid = widgets.IntSlider(value=data.shape[1]//2,
                                               min=0,
                                               max=data.shape[1]-1,
                                               step=1))

interactive(children=(IntSlider(value=375, description='sid', max=750), Output()), _dom_classes=('widget-inter…

<function __main__.plot_projections(sid=1)>

# Plot sinograms

In [6]:
def plot_sinogram(sid=1):
    plt.imshow(data[sid],cmap='gray')
interact(plot_sinogram, sid = widgets.IntSlider(value=data.shape[0]//2,
                                               min=0,
                                               max=data.shape[0]-1,
                                               step=1))

interactive(children=(IntSlider(value=512, description='sid', max=1023), Output()), _dom_classes=('widget-inte…

<function __main__.plot_sinogram(sid=1)>

# Create a class for reconstruction. Minus logarithm is turned off, dark=0, flat=1 since the inital data is already preprocessed.

In [7]:
cl = GPURecRAM.for_data_like(
    data=data,
    dark=dark,
    flat=flat,
    ncz=8,  # chunk size for GPU processing (multiple of 2), 
    rotation_axis=center,  # rotation center
    dtype="float32",  # computation type, note  for float16 n should be a power of 2
    fbp_filter='parzen',
    minus_log=False
)

# Reconstruct by Tomocupy (the non-command-line version)

In [8]:
obj = cl.recon_all(data, dark, flat, theta)

chunk 128/128 |  |████████████████████████████████████████| 100.0% 


In [9]:
def plot_rec(sid=1):
    plt.imshow(obj[sid],cmap='gray',vmin=-0.005,vmax=0.01)
interact(plot_rec, sid = widgets.IntSlider(value=obj.shape[0]//2,
                                               min=0,
                                               max=obj.shape[0]-1,
                                               step=1))

interactive(children=(IntSlider(value=512, description='sid', max=1023), Output()), _dom_classes=('widget-inte…

<function __main__.plot_rec(sid=1)>

# Reproject data

In [10]:
data_reproj = cl.proj_all(obj,theta)

chunk 128/128 |  |████████████████████████████████████████| 100.0% 


In [11]:
def plot_projections(sid=1):
    plt.figure(figsize=(15,10))
    plt.subplot(1,2,1)
    plt.title('initial data')
    plt.imshow(data[:,sid],cmap='gray')
    plt.subplot(1,2,2)
    plt.title('reprojected data')
    plt.imshow(data_reproj[:,sid],cmap='gray')
interact(plot_projections, sid = widgets.IntSlider(value=data.shape[1]//2,
                                               min=0,
                                               max=data.shape[1]-1,
                                               step=1))

interactive(children=(IntSlider(value=375, description='sid', max=750), Output()), _dom_classes=('widget-inter…

<function __main__.plot_projections(sid=1)>

In [12]:
def plot_sinogram(sid=1):
    plt.figure(figsize=(15,10))
    plt.subplot(1,2,1)
    plt.title('initial data')
    plt.imshow(data[sid],cmap='gray')
    plt.subplot(1,2,2)
    plt.title('reprojected data')
    plt.imshow(data_reproj[sid],cmap='gray')    
interact(plot_sinogram, sid = widgets.IntSlider(value=data.shape[0]//2,
                                               min=0,
                                               max=data.shape[0]-1,
                                               step=1))

interactive(children=(IntSlider(value=512, description='sid', max=1023), Output()), _dom_classes=('widget-inte…

<function __main__.plot_sinogram(sid=1)>

# Reconstruct the object by using reprojected data

## We can use the ramp filter instead of the parzen filter since we want to keep high frequencies in images. So we recreate the reconstruction class

In [13]:
cl = GPURecRAM.for_data_like(
    data=data,
    dark=dark,
    flat=flat,
    ncz=8,  # chunk size for GPU processing (multiple of 2), 
    rotation_axis=center,  # rotation center
    dtype="float32",  # computation type, note  for float16 n should be a power of 2
    fbp_filter='ramp',
    minus_log=False
)

In [14]:
obj_reproj = cl.recon_all(data_reproj, dark, flat, theta)

chunk 128/128 |  |████████████████████████████████████████| 100.0% 


## Adjust scaling

In [15]:
obj_reproj*=np.linalg.norm(obj)/np.linalg.norm(obj_reproj)

In [16]:
def plot_rec(sid=1):
    plt.figure(figsize=(15,10))
    plt.subplot(1,2,1)
    plt.title('initial reconstruction')
    plt.imshow(obj_reproj[sid],cmap='gray',vmin=-0.005,vmax=0.01)
    plt.colorbar(orientation='horizontal')
    plt.subplot(1,2,2)
    plt.title('reconstruction after reprojection')
    plt.imshow(obj[sid],cmap='gray',vmin=-0.005,vmax=0.01)
    plt.colorbar(orientation='horizontal')
interact(plot_rec, sid = widgets.IntSlider(value=obj.shape[0]//2,
                                               min=0,
                                               max=obj.shape[0]-1,
                                               step=1))

interactive(children=(IntSlider(value=512, description='sid', max=1023), Output()), _dom_classes=('widget-inte…

<function __main__.plot_rec(sid=1)>

## Plot difference, note colorbar

In [17]:
def plot_rec(sid=1):
    plt.figure(figsize=(15,10))
    plt.imshow(obj[sid]-obj_reproj[sid],cmap='gray',vmin=-0.0002,vmax=0.0002)    
    plt.colorbar(orientation='horizontal')
interact(plot_rec, sid = widgets.IntSlider(value=obj.shape[0]//2,
                                               min=0,
                                               max=obj.shape[0]-1,
                                               step=1))

interactive(children=(IntSlider(value=512, description='sid', max=1023), Output()), _dom_classes=('widget-inte…

<function __main__.plot_rec(sid=1)>

# Conclusion. 1. Data after reprojection give the same reconstruction as the initial data. But reprojected data is less noisy, looks smoother (not many hig frequency components) and might be easier to compress. 2. The reconstructed object and its reprojected data can be considered as a synthetic data for further compression tests. We have 1-1 mapping between the object and the data.